# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [4]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd



In [5]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [10]:
# Your code here; use a subquery. No join is necessary 
q = """SELECT customerNumber, contactLastName, contactFirstName
       FROM customers
       WHERE customerNumber IN (SELECT customerNumber 
                                FROM orders 
                                WHERE orderDate = '2003-01-31')
       ;"""
cur.execute(q).fetchall()

df = pd.read_sql(q, conn)
print(f'{len(df)} Results')
df

1 Results


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [30]:
# Your code here
q = """SELECT p.productName, COUNT(od.orderNumber) as total_num_orders, SUM(od.quantityOrdered) as total_items_ordered 
       FROM products AS p
       JOIN orderdetails AS od USING('productCode')
       GROUP BY p.productName
       ORDER by total_items_ordered DESC
       ;"""

cur.execute(q).fetchall()
df = pd.read_sql(q, conn)
print(df.shape)
df

(109, 3)


,productName,total_num_orders,total_items_ordered
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [39]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q = """SELECT p.productName, COUNT(DISTINCT o.customerNumber) as totalNumPurchasers
       FROM products AS p
       JOIN orderdetails AS od USING(productCode)
       JOIN orders AS o USING(orderNumber)
       GROUP BY p.productName
       ORDER BY totalNumPurchasers DESC
       ;"""

cur.execute(q).fetchall()

df = pd.read_sql(q, conn)
print(df.shape)
df

(109, 2)


,productName,totalNumPurchasers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [70]:
# Your code here
emp_q = """SELECT e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
       FROM employees AS e
       JOIN offices AS o USING(officeCode)
       ;"""

#cur.execute(emp_q).fetchall()

prod_q = """SELECT p.productCode
            FROM products AS p
            JOIN orderdetails AS od USING(productCode)
            JOIN orders AS o USING(orderNumber)
            GROUP BY p.productCode
            HAVING COUNT(DISTINCT od.orderNumber < 20)
            ;"""
#cur.execute(prod_q).fetchall()


q = """"SELECT DISTINCT employeeNumber, firstName, lastName, city, officeCode
            FROM employees e
            JOIN offices o USING(officeCode)
            JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders USING(customerNumber)
            JOIN orderdetails USING(orderNumber)
            WHERE productCode IN (SELECT productCode
                                         FROM products
                                         JOIN orderdetails USING(productCode)
                                         JOIN orders USING(orderNumber)
                                         GROUP BY productCode
                                         HAVING COUNT(DISTINCT customerNumber < 20)
                                  );"""

cur.execute(q).fetchall()

OperationalError: unrecognized token: ""SELECT DISTINCT employeeNumber, firstName, lastName, city, officeCode
            FROM employees e
            JOIN offices o USING(officeCode)
            JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders USING(customerNumber)
            JOIN orderdetails USING(orderNumber)
            WHERE productCode IN (SELECT productCode
                                         FROM products
                                         JOIN orderdetails USING(productCode)
                                         JOIN orders USING(orderNumber)
                                         GROUP BY productCode
                                         HAVING COUNT(DISTINCT customerNumber < 20)
                                  );"

In [71]:
## solution  :/  
q = """SELECT DISTINCT employeeNumber, officeCode, o.city, firstName, lastName
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber < 20)
                                            );"""

cur.execute(q).fetchall()

[(1216, 2, 'Boston', 'Steve', 'Patterson'),
 (1504, 7, 'London', 'Barry', 'Jones'),
 (1286, 3, 'NYC', 'Foon Yue', 'Tseng'),
 (1370, 4, 'Paris', 'Gerard', 'Hernandez'),
 (1401, 4, 'Paris', 'Pamela', 'Castillo'),
 (1323, 3, 'NYC', 'George', 'Vanauf'),
 (1621, 5, 'Tokyo', 'Mami', 'Nishi'),
 (1501, 7, 'London', 'Larry', 'Bott'),
 (1165, 1, 'San Francisco', 'Leslie', 'Jennings'),
 (1337, 4, 'Paris', 'Loui', 'Bondur'),
 (1702, 4, 'Paris', 'Martin', 'Gerard'),
 (1611, 6, 'Sydney', 'Andy', 'Fixter'),
 (1166, 1, 'San Francisco', 'Leslie', 'Thompson'),
 (1612, 6, 'Sydney', 'Peter', 'Marsh'),
 (1188, 2, 'Boston', 'Julie', 'Firrelli')]

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [85]:
# Your code here
q = """SELECT employeeNumber, firstName, lastName, COUNT(customerNumber) as numGoodCreditCustomers
           FROM employees e
           JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
           GROUP BY e.employeeNumber
           HAVING AVG(creditLimit) > 15000
           ;"""

cur.execute(q).fetchall()

df = pd.read_sql(q, conn)
print(df.shape)
df


(15, 4)


,employeeNumber,firstName,lastName,numGoodCreditCustomers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!